# 特殊な加工・可視化を行なう10本 Knock

## 大容量CSV data を扱ってみよう
容量無制限に処理できるものではない為、Memory容量との兼ね合いなどを考慮しつつ工夫する必要がでてくる、

※時系列Data を大容量Data と仮定して使用し演習を実施。Data が小さくても Code の書き方と結果を確認し Image を掴む。

In [2]:
# File を読み込む
import os

import pandas as pd

df = pd.read_csv('data/chapter8/person_count_out_0001_2021011509.csv')
df

,id,place,receive_time,sensor_num,in1,out1,state1,in2,out2,state2
0,0,1,2021-01-15 09:00:00.144,2,508,73,0,73,508,0
1,1,1,2021-01-15 09:00:01.146,2,508,73,0,73,508,0
2,2,1,2021-01-15 09:00:02.161,2,508,73,0,73,508,0
3,3,1,2021-01-15 09:00:03.176,2,508,73,0,73,508,0
4,4,1,2021-01-15 09:00:04.192,2,508,73,0,73,508,0
...,...,...,...,...,...,...,...,...,...,...
3535,3535,1,2021-01-15 09:59:55.054,2,782,156,0,156,782,0
3536,3536,1,2021-01-15 09:59:56.07,2,782,156,0,156,782,0
3537,3537,1,2021-01-15 09:59:57.085,2,782,156,0,156,782,0
3538,3538,1,2021-01-15 09:59:58.101,2,782,156,0,156,782,0


仮に Data数が 100万~1000万以上の Data になると pandas.read_csv() で一括で読み込もうとすると OOM Error で Program が落ちると想定される。
※ OOM Error: Memory不足による Error

In [3]:
# OOM Error を回避する為に引数で桁数を指定して読み込む
for df in pd.read_csv('data/chapter8/person_count_out_0001_2021011509.csv', chunksize=512):
    print(df.shape)

(512, 10)
(512, 10)
(512, 10)
(512, 10)
(512, 10)
(512, 10)
(468, 10)


`chunksize=` 明示的に指定すると指定した行数ごとに CSV File を読み込む

In [4]:
# 読み込んだ Dataに対して何らかの処理を行なった上で、別File に保存してみる
i = 0
for df in pd.read_csv('data/chapter8/person_count_out_0001_2021011509.csv', chunksize=64):  # Chunksize を指定して読み込み
    df['processed_per_chunk'] = True  # column を追加
    df.to_csv('data/chapter8/processed_big_data.csv', mode='a', index_label=False, header=i == 0)  # 新しい File に保存していく
    # media='a' と指定することで同じ File に対して追記して保存する。（※'a': append の頭文字）
    # header=i == 0 とすることで i == 0 の時だけ header ありで出力するようにしている

    i += 1

In [5]:
# 結果の確認
from IPython.display import display

df = pd.read_csv('data/chapter8/processed_big_data.csv')
display(df)

,id,place,receive_time,sensor_num,in1,out1,state1,in2,out2,state2,processed_per_chunk
0,0,1,2021-01-15 09:00:00.144,2,508,73,0,73,508,0,True
1,1,1,2021-01-15 09:00:01.146,2,508,73,0,73,508,0,True
2,2,1,2021-01-15 09:00:02.161,2,508,73,0,73,508,0,True
3,3,1,2021-01-15 09:00:03.176,2,508,73,0,73,508,0,True
4,4,1,2021-01-15 09:00:04.192,2,508,73,0,73,508,0,True
...,...,...,...,...,...,...,...,...,...,...,...
3535,3535,1,2021-01-15 09:59:55.054,2,782,156,0,156,782,0,True
3536,3536,1,2021-01-15 09:59:56.07,2,782,156,0,156,782,0,True
3537,3537,1,2021-01-15 09:59:57.085,2,782,156,0,156,782,0,True
3538,3538,1,2021-01-15 09:59:58.101,2,782,156,0,156,782,0,True


## JSON 形式の File を扱ってみよう

In [6]:
pd.read_json('data/chapter8/column_oriented.json')

,id,value
0,1,1
1,2,10
2,3,100


In [7]:
# Pandas で読み込む前の状態を確認
!type data\chapter8\column_oriented.json

{"id":{"0":1,"1":2,"2":3},"value":{"0":1,"1":10,"2":100}}


列指向の JSON File という

In [8]:
!type data\chapter8\index_oriented.json

{"0":{"id":1,"value":1},"1":{"id":2,"value":10},"2":{"id":3,"value":100}}


Index指向の JSON File である

In [9]:
pd.read_json('data/chapter8/index_oriented.json')

,0,1,2
id,1,2,3
value,1,10,100


縦横が逆になっている

In [10]:
pd.read_json('data/chapter8/index_oriented.json', orient='index')
# 引数 orient='index' を指定することで Index指向のJSON File が正しく読み込める

,id,value
0,1,1
1,2,10
2,3,100


In [11]:
# Table 指向な構造の JSON File（※ RDB の File を JSON File に dump する際に見られる構造）
!type data\chapter8\table_oriented.json

{"schema":{"fields":[{"name":"index","type":"integer"},{"name":"id","type":"integer"},{"name":"value","type":"integer"}],"primaryKey":["index"],"pandas_version":"0.20.0"},"data":[{"index":0,"id":1,"value":1},{"index":1,"id":2,"value":10},{"index":2,"id":3,"value":100}]}


In [12]:
# そのまま Pandas で読み込んでみる
pd.read_json('data/chapter8/table_oriented.json')

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

Error になる

In [13]:
pd.read_json('data/chapter8/table_oriented.json', orient='table')
# 引数 orient='table' を指定することで Table指向の JSON File も読み込める

,id,value
0,1,1
1,2,10
2,3,100


JSON 形式には様々な構造が存在するため、読み込む際は注意が必要。
※今回の演習で登場した構造以外にも pandas.read_json() で読み込める構造がある。

## Web からの Data を取得してみよう
Web site に対して Request を http送信した際、Server が Client に結果を送信する Case では、JSON形式が使われる場合が多い。
上記の Case の場合の Data の扱う

In [14]:
# worldtimeapi.org という Site に http request を送信し、東京の時刻等の情報を取得して、Contents を表示する
import requests  # http request を扱う Package として requests を選択

response = requests.get('https://worldtimeapi.org/api/timezone/Asia/Tokyo')
response.content

b'{"abbreviation":"JST","client_ip":"60.71.122.177","datetime":"2022-06-11T07:31:27.342018+09:00","day_of_week":6,"day_of_year":162,"dst":false,"dst_from":null,"dst_offset":0,"dst_until":null,"raw_offset":32400,"timezone":"Asia/Tokyo","unixtime":1654900287,"utc_datetime":"2022-06-10T22:31:27.342018+00:00","utc_offset":"+09:00","week_number":23}'

Response が JSON形式の Data であることを確認。

In [15]:
# JSONのままでは扱いずらいため dict型に変換
result = response.json()  # Response.json() で dict型に変換可能
result

{'abbreviation': 'JST',
 'client_ip': '60.71.122.177',
 'datetime': '2022-06-11T07:31:27.342018+09:00',
 'day_of_week': 6,
 'day_of_year': 162,
 'dst': False,
 'dst_from': None,
 'dst_offset': 0,
 'dst_until': None,
 'raw_offset': 32400,
 'timezone': 'Asia/Tokyo',
 'unixtime': 1654900287,
 'utc_datetime': '2022-06-10T22:31:27.342018+00:00',
 'utc_offset': '+09:00',
 'week_number': 23}

Key と値の関係について視認性が向上した。

In [16]:
# Pandas.Series に変換してみる
pd.Series(result)  # Pandas.Series() に引数を渡すことで Series に変換

abbreviation                                 JST
client_ip                          60.71.122.177
datetime        2022-06-11T07:31:27.342018+09:00
day_of_week                                    6
day_of_year                                  162
dst                                        False
dst_from                                    None
dst_offset                                     0
dst_until                                   None
raw_offset                                 32400
timezone                              Asia/Tokyo
unixtime                              1654900287
utc_datetime    2022-06-10T22:31:27.342018+00:00
utc_offset                                +09:00
week_number                                   23
dtype: object

In [17]:
# 取得結果を保存しておく
import json  # Json Libray を import

with open('dump/response.json', mode='w') as f:  # with open に出力File 名を記載して mode='w'(書き込みMode)で Open.
    json.dump(result, f)  # Json.dump() で File に内容を書き込み

In [18]:
# 定期的に Request を実行し、結果を１つの File に追記する
import time  # time Library を import

for _ in range(4):  # renge(4)で４回繰り返し処理
    response = requests.get('https://worldtimeapi.org/api/timezone/Asia/Tokyo')
    with open('dump/response.txt', mode='a') as f:  # mode='a' で追記Mode で File を Open
        res = response.json()  # 変数に response を Response.json() で dict型で格納
        f.write(f"{json.dumps(res)}\n")  # File に変数に格納した Data を出力していく
    time.sleep(1)  # １秒置く

In [19]:
# 出力した Data を確認
!type dump\response.txt

{"abbreviation": "JST", "client_ip": "121.119.11.5", "datetime": "2022-06-08T15:38:48.264271+09:00", "day_of_week": 3, "day_of_year": 159, "dst": false, "dst_from": null, "dst_offset": 0, "dst_until": null, "raw_offset": 32400, "timezone": "Asia/Tokyo", "unixtime": 1654670328, "utc_datetime": "2022-06-08T06:38:48.264271+00:00", "utc_offset": "+09:00", "week_number": 23}
{"abbreviation": "JST", "client_ip": "121.119.11.5", "datetime": "2022-06-08T15:38:49.319286+09:00", "day_of_week": 3, "day_of_year": 159, "dst": false, "dst_from": null, "dst_offset": 0, "dst_until": null, "raw_offset": 32400, "timezone": "Asia/Tokyo", "unixtime": 1654670329, "utc_datetime": "2022-06-08T06:38:49.319286+00:00", "utc_offset": "+09:00", "week_number": 23}
{"abbreviation": "JST", "client_ip": "121.119.11.5", "datetime": "2022-06-08T15:38:50.398317+09:00", "day_of_week": 3, "day_of_year": 159, "dst": false, "dst_from": null, "dst_offset": 0, "dst_until": null, "raw_offset": 32400, "timezone": "Asia/Tokyo", 

## Config File を扱ってみよう

In [20]:
# yaml File の中身を出力し構造を確認
!type data\chapter8\config.yml

dataset:
  name: pseudo
  path: data/images_by_py/
use_gpu: true


In [21]:
# File を読み込む
import yaml  # PyYaml Libray を import（※ PyYaml Library を要 pipenv install）

with open('data/chapter8/config.yml', mode='r') as f:  # with open で mode='r'(読み込みMode)で Open.
    config = yaml.safe_load(f)  # yaml.safe_load() で Data を読み込み変数に格納
config  # 変数を出力

{'dataset': {'name': 'pseudo', 'path': 'data/images_by_py/'}, 'use_gpu': True}

Data を確認すると、Json形式で保持されいている。

In [22]:
# toml File の構造を確認
!type data\chapter8\config.toml

use_gpu = true

[dataset]
name = "pseudo"
path = "data/images_by_py/"


In [23]:
import toml  # toml Library を import ※ pipenv install toml 要必要

with open('data/chapter8/config.toml', mode='r') as f:  # File を mode='r'(読み込み mode)で開く
    config = toml.load(f)  # toml.load() 関数で内容を読み込み変数に格納

config

{'use_gpu': True, 'dataset': {'name': 'pseudo', 'path': 'data/images_by_py/'}}

Json 形式で Data が保持されていることを確認。

## 動画File を音声File へ変換してみよう
音声 Data を用意する手法のひとつとして、動画 File から音声 Data に変換してみる

In [24]:
from moviepy.editor import VideoFileClip  # moviepy Package から VideoFileClip を import ※要 pipenv install moviepy

video_clip = VideoFileClip('data/chapter8/sample_video.mp4')  # 動画File から VideoFileClip data を作成し変数に格納
video_clip.audio.write_audiofile(
    'dump/audio_by_py.mp3')  # VideoFileClip.audio.write_audiofile() 関数で mp4形式の動画File から mp3 形式の音声File に変換して出力

MoviePy - Writing audio in dump/audio_by_py.mp3


MoviePy - Done.


## 動画Flie を画像File へ分割してみよう
実用を考えて、動画File を複数の画像File に分割する処理を行なう。

In [2]:
import cv2  # 画像処理を行なう Library を import
from tqdm import trange  # 処理時間を表示してくれる Library を import. ※ 処理時間が長くかかるような時はお薦め
import os  # OS の操作を Python で扱う為の Library を import

cap = cv2.VideoCapture('data/chapter8/sample_video.mp4')  # 動画File を読み込み変数に格納
img_dir = 'dump/images_by_py'  # 処理の結果を出力して保存する為の Directory の Path を変数に格納
os.makedirs(img_dir, exist_ok=1)  # 上記、Directory を作成。 exist_ok=1 で指定した　Directory が既設の場合でも Error をはかないように設定
n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # 動画File の Frame数を取得し変数に格納

for i in trange(n):  # Frame数で Loop処理。trange()で囲むことで進捗状況を可視化してくれる
    success, img = cap.read()  # Frame単位で画像を読み込み
    if not success:
        continue  # 読み込みに失敗した場合は何もしないように指定
    cv2.imwrite(f"{img_dir}/{i:04}.png", img)  # 読み込みに成功した時に、Frame番号４桁で設定し出力

100%|██████████| 389/389 [00:03<00:00, 102.83it/s]


In [4]:
print(os.listdir(img_dir))  # 出力結果を確認

['0000.png', '0001.png', '0002.png', '0003.png', '0004.png', '0005.png', '0006.png', '0007.png', '0008.png', '0009.png', '0010.png', '0011.png', '0012.png', '0013.png', '0014.png', '0015.png', '0016.png', '0017.png', '0018.png', '0019.png', '0020.png', '0021.png', '0022.png', '0023.png', '0024.png', '0025.png', '0026.png', '0027.png', '0028.png', '0029.png', '0030.png', '0031.png', '0032.png', '0033.png', '0034.png', '0035.png', '0036.png', '0037.png', '0038.png', '0039.png', '0040.png', '0041.png', '0042.png', '0043.png', '0044.png', '0045.png', '0046.png', '0047.png', '0048.png', '0049.png', '0050.png', '0051.png', '0052.png', '0053.png', '0054.png', '0055.png', '0056.png', '0057.png', '0058.png', '0059.png', '0060.png', '0061.png', '0062.png', '0063.png', '0064.png', '0065.png', '0066.png', '0067.png', '0068.png', '0069.png', '0070.png', '0071.png', '0072.png', '0073.png', '0074.png', '0075.png', '0076.png', '0077.png', '0078.png', '0079.png', '0080.png', '0081.png', '0082.png', '00

## PowerPoint や Word File を読み込んでみよう
Office は、身近な Data だが CPU data として取り扱うには Data品質が悪い。
できるだけ取り扱いは控えるべきだが人間には取り扱いやすく利用率も高いため、Data資源として活用したい Needs は高い。
活用には工夫が必要な Case が多い。

In [1]:
import pptx  # PowerPoint を Python で扱うためのの Module を import. ※要 `pipenv install python-pptx`

pptx_data = pptx.Presentation('data/chapter8/サンプル_PowerPoint.pptx')  # PowerPoint data を読み込み変数に格納
len(pptx_data.slides)  # Slide の page数を確 ※ Presentation.slides で Page毎の情報を取得できる。

2

In [2]:
sld_0 = pptx_data.slides[0]  # １page目の Slide情報を読み込み変数に格納
shp_sld_0 = sld_0.shapes  # Page にいくつ情報があるか読み込み表示
len(shp_sld_0)  # Slide情報がいくつか確認

3

In [3]:
print(shp_sld_0[0].text)  # Slide 1 page目の１つ目の情報にある Text data を抽出
print(shp_sld_0[0].has_text_frame)  # 上記の情報が Text data か真偽を確認

サンプルテキスト Font 18
True


In [5]:
texts = []  # 空の list を準備する
for slide in pptx_data.slides:  # Slide を Page毎に loop する
    for shape in slide.shapes:  # 各Slide page から情報を抽出
        if shape.has_text_frame:  # 対象の情報に Text は含まれているか判別
            texts.append(shape.text)  # Text data であれば準備した list に追加
print(texts)  # list を確認

['サンプルテキスト Font 18', 'サンプルテキスト\nFont 28', 'サンプル', '２枚目サンプルテキスト Font 18', '２枚目サンプルテキスト\nFont 28', '', '']


In [7]:
import docx  # Word を Python で扱ための Module を import ※ 要`pipenv install python-docx`

docx_data = docx.Document('data/chapter8/サンプル_Word.docx')  # Word data を読み込み変数に格納
len(docx_data.paragraphs)  # Word Data の Paragraph がいくつかるか確認。※ Word data は Paragraph という Data単位で取り扱う

3

In [8]:
docx_data.paragraphs[0].text  # １行目の Text を確認

'これは、サンプルテキストです。そして、これが一つめの段落になっています。いろいろ読み込んでいきましょう。'

In [9]:
texts = []  # 空の list を準備
for paragraph in docx_data.paragraphs:  # Word data の Paragraphs を loop処理
    texts.append(paragraph.text)  # list に Paragraph data を追加していく
print(texts)  # list に追加された Data を確認

['これは、サンプルテキストです。そして、これが一つめの段落になっています。いろいろ読み込んでいきましょう。', '続いて、これが二つ目の段落になっています。', 'これが三つめの段落です。']
